# loading packages 

In [ ]:


#bioconductor
library("GenomicRanges")
library("DESeq2")
library("ACME")
library("GEOquery")
library("EnhancedVolcano")

#R 
library("stringr")
library("dplyr")
library("parallel")
library("rmarkdown")
library("knitr")
library("ggfortify")
library("data.table")
library("ggrepel")

#define the input path
filepath <- c("/public/codelab/omics-workshop/OmicsWorkshopVignettes/03_BulkRNAATAC_KithXiang/data")


# ATAC-seq

An ATAC-seq analysis is very similar to RNA-seq.  Once we have the count matrix, almost
all the steps are the same.  

# Processesing from scratch

The preparation of the count matrix from the fastq files is what takes the most
time and effort and though it's possible to do this all on a laptop, 
it's usually better to do the computationally heavy parts on the HPC.

A typical ATAC-seq pipeline involves the following steps.

## Quality control & adapter trimming

Just like RNA-seq, we should run QC on the reads and trim adapters before going further with the processing.

[*fastqc*](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/), 
[*trim_galore*](https://github.com/FelixKrueger/TrimGalore)


## Alignment

Alignment of ATACseq data is in a sense easier than RNAseq, because we don't have to
consider the splicing of reads over exons.

[*bowtie2*](https://bowtie-bio.sourceforge.net/bowtie2/manual.shtml) is a popular option for this step
and can be run on the HPC or even a laptop.

Its main input parameters are the location of the indexed reference genome and the fastq files, and
the default options work well for ATACseq data.  

A quirk of this program is that it has trouble reading zipped files, so you either have to unzip
your compressed fastq files beforehand, or better yet, use 
[process substitution](https://en.wikipedia.org/wiki/Process_substitution)
at the command line to avoid
temp files.  The alignments come out in SAM format, and if you want binary output, 
you can pipe it to [*samtools*](http://www.htslib.org/doc/samtools.html) to save
as a BAM. 

## Marking PCR duplicates

Steps should be taken to ensure that we'll be using one single read per physical fragment of our ATACseq library.
Multiple copies of reads can come from PCR or optical duplication and we need a way to mark them, 
and to make sure we're using the one with the best quality.

The [*MarkDuplicates*](https://gatk.broadinstitute.org/hc/en-us/articles/360037052812-MarkDuplicates-Picard-) 
feature of the [*picard-tools*](https://broadinstitute.github.io/picard/) software suite does this nicely, 
adding a flag to each read of the bam telling if it should be used or not.  As this is such 
a common step, most downstream programs take this
BAM [*flag*](http://broadinstitute.github.io/picard/explain-flags.html) 
into consideration, so manually removing the the duplicates usually isn't required.

Before running `MarkDuplicates`, you'll need to sort your bam file.
After running `MarkDuplicates`, it's a good idea to index your bam.  This speeds up
query operations and lets downstream programs jump to locations without having to read in
the entire file.

## Peak detection

Once the reads have been mapped into a sorted BAM file, figuring out where the reads fall into peaks can be done 
with a program called [*Model-based Analysis of ChIP-seq (MACS)*](https://hbctraining.github.io/Intro-to-ChIPseq/lessons/05_peak_calling_macs.html).

It was originally designed for finding trascription binding sites in ChIP-seq experiments, but there
are options that allow it to be run without having an input control sample.  It uses a
technique called *lamda shifting* to estimate background signal in the local region around a potential peak,
and a Poisson model is used to determine its significance. 


The main output from the program are the locations of the peaks that were detected in various
formats(bed,xls).

## Running the pipeline

The steps described above are the most computationally intensive, so running them 
on the HPC is recommended.  The script below shows an example of how to process a set of trimmed
fastq files with the programs available on the cluster using the 
`sbatch` command.



### slurm_peakPipeline.sh

## Enumerating peaks

Once we have the bam(aligned reads) and xls(detected peaks) files, we can
count the number of reads falling in each peak region for each sample.

### Merging peaks

First we get a composite set of peaks merged from the ranges of all of our samples.
Peak detection is run on all samples individually, then those results are merged
together and collected into a single set that is the union of all the ranges.

In [ ]:

#load in the peak files
pfiles = list.files(path=paste0(filepath,"/macsResults"), recursive=T, pat="^.*xls$", full=T)
peaks = lapply(pfiles, read.table, header=T)

#contruct genomic ranges of the peak summits +/- 200bp
ranges = lapply(peaks, function(peak){
	GRanges(seqnames = peak[,1], 
        strand=rep("*", nrow(peak)), 
            IRanges(start = peak[,2], end = peak[,3]))
})
peaks.reduced = reduce(do.call(c, ranges))

#save genomic range as a bed file
#https://www.biostars.org/p/89341/
grange2bed <- function(gr, bfile="foo.bed"){

	df <- data.frame(seqnames=seqnames(gr),
		starts=start(gr)-1,
		ends=end(gr),
		names=c(rep(".", length(gr))),
		scores=c(rep(".", length(gr))),
		strands=strand(gr))

	write.table(df, file=bfile, quote=F, sep="\t", row.names=F, col.names=F)
}

grange2bed(peaks.reduced, bfile="peaks_stat.bed")



### Counting reads

For every sample, we use these ranges to count the number of reads falling within the peaks.

This can be done with a program called
[*bedmap*](https://bedops.readthedocs.io/en/latest/content/reference/statistics/bedmap.html)
from the [*BEDOPS*](https://bedops.readthedocs.io/en/latest/)
toolkit.

bedmap requires the input to be in .bed format, so in the script below the bam file is converted on the fly
with process substitution and the output is saved in a txt file.

This step can take a while so these precomputed count files have been included in
the vingette's `data/` folder.

## Loading peaks into R
The previous steps show us how to create count files for each sample showing the read coverages of every peak range.  
They are simple .txt files and can be loaded into R very quickly.


In [ ]:


pfiles = list.files(path=paste0(filepath,"/counts_atac"), pat="*counts", full=T)

#load in 1 file to have a look at the format
x = read.csv(pfiles[1], sep="\t", header=F)
head(x)

In [ ]:
#process all the count files
pcounts = lapply(pfiles, function(file){
    #print(file)
	x = read.csv(file, sep="\t", header=F)
    #extract the number that appears after the *| in the 6th column of the input
	hits = as.numeric(gsub(x[,6], pat="\\*\\|", rep=""))
	y = data.frame(chr=x[,1], start=x[,2], end=x[,3], counts=hits)
	y
})
#bind the counts together as columns in a matrix
peakdata = do.call(cbind, lapply(pcounts, function(a){
	a$counts
}))
#give the columns better looking names. 
#grab the strings between the last "/" and ".counts" 
colnames(peakdata) = gsub(pfiles, pat=".*/(.*).counts", rep="\\1")


#peak locations
peaks = pcounts[[1]][,1:3]
head(peaks)

In [ ]:
#make a descriptive string ID for the peak 
peaknames = sapply(1:nrow(peaks), function(i){
	paste0(peaks[i,1:3], collapse="_")
})
rownames(peakdata) = peaknames
cbind(head(rownames(peakdata)))

In [ ]:

#get rid of NA counts
ix2 = which(apply(peakdata, 1, function(a){
    !any(is.na(a))
}))
peakdata=peakdata[ix2,]

dim(peakdata)
head(peakdata)


# Running Deseq
From this point on, the files are processed and we're working with a count matrix that
 can be analyzed in the usual way.  The main difference is that the features are now peak id's
instead of genes.  This doesn't change the way DESeq2 handles the data, but downstream analysis
will need to be adjusted, usually by appending info of the nearest gene and filtering peaks 
too far away from points of interest.

We already have our count matrix ready, now we need to create the phenotype data.frame.
You might find it easier to prepare this in an excel spreadsheet first then load it into R.


In [ ]:
info <- colnames(peakdata)
comp1 <- rep(NA, length(info))
comp1[1:2] <- "ctrl"
comp1[3:4] <- "drug_treated"
coldata <- data.frame(info, comp1)

#always make sanity checks with your own data
#the row order of coldata needs to match the column order of peakdata
all(coldata$info == colnames(peakdata))

In [ ]:

head(coldata)


In [ ]:
#run the analysis
dds <- DESeqDataSetFromMatrix(countData = peakdata,
								  colData = coldata,
								  design = ~ comp1)
dds <- DESeq(dds)
res <- results(dds)
res <- as.data.frame(res)

head(res, 50)

## A generalized function

If you're going to be working on more than one set of samples, it makes
sense to wrap up the DESEq2 analysis and any post processing routines
into a function which can be applied to each comparison group.

In the function below, we're calling DESeq2 as usual, then tacking on
peak info and the raw & normalized peak counts to the resulting data.frame,
then saving the output to a txt file.  


In [ ]:


#run DESeq, add peak info, sort and write as a csv.
#countdata: a matrix of integer counts, (nPeak x nSamples)
#coldata: a data.frame with info on the samples (nSamples x nVariables)
#ix: the indices of the samples you want to use in the analysis.
#formula: shows which columns of coldata you want to use in the analysis
#resFile: is where to save the result file
runDE <- function(countdata, coldata, ix, formula = formula("~ ep300"), resFile){

	dds = DESeqDataSetFromMatrix(countData = countdata[,ix],
								  colData = coldata[ix,],
								  design = formula)
	dds <- updateObject(dds)
	dds = DESeq(dds)
	res = results(dds)
	y = data.frame(res)

    #break the peak identifier into parts
    pos = data.frame(pid = rownames(y), str_split_fixed(rownames(y), "_", 3))
    colnames(pos)[2:4] = c("chrom", "start", "stop")
    pos$start = as.numeric(pos$start)
    pos$stop = as.numeric(pos$stop)
    pos$mid = round((pos$stop+pos$start)/2)
	y = cbind(pos, y)

    #add peak counts
	diffexp = merge(y, countdata[,ix], by.x=1, by.y=0, all=T)

	#append the normalized counts to the matrix
	norm.counts = counts(dds, normalized=T)
	colnames(norm.counts) = paste0("norm_",dds$info)
	y = merge(diffexp, norm.counts, by.x=1, by.y=0, all=T)

    #sort results by fold change
	y = y[order(y[,"log2FoldChange"]),]

    #add a chromosomal position to make IGV navigation easier
    y$chrpos = paste0(y$chrom, ":", y$start, "-", y$stop)

	write.table(file=resFile, x=y, row.names=F, sep="\t")

    #also return the results
    y
}

## Calling the function

Once the function is written, it's much easier to run different comparison groups
just by calling it with different parameters.  It's not too helpful in this example, but imagine
you had a larger cohort of samples all treated with different drugs and experimental
conditions.  

In [ ]:
#use the non-NA samples in comp1
ix = which(!is.na(coldata$comp1))
#or you can do it like
#ix = 1:4

#run the analysis
res = runDE(peakdata, coldata, ix, formula("~ comp1"), "de_CtrlVsDrug.txt")

#view some of the results
head(res, 50)


## Annotating peak locations

You will most likely need to understand where the peaks are in relation to the genes of
your reference genome.  The `ACME` bioconductor package has an easy to use function that
can do this, which we can wrap in an `mclapply` call to run it in parallel over multiple cores.

The function below tacks on information of the closest gene to each peak, 
and includes the distance to the transcription start site.

For illustrative purposes and to make this run faster, 
we'll just work with the regions that have padj < 0.05.

In [ ]:
appendClosestGeneInfo <- function(x, chr=1, pos = 2, genome="hg19", numCores = 1){
	res = mclapply(1:nrow(x), function(i){
		if (i %% 1000 == 0){
			print(i)
		}
		genes = findClosestGene(x[i,chr],x[i, pos],genome)
		#if there's more than one, just take the first
		data.frame(genes[1,])
	}, mc.cores = numCores)

	x.res = bind_rows(res)
	cbind(x.res, x)
}

#how many peaks show significant DE?
ix.sig = which(res$padj < 0.05)
length(ix.sig)

In [ ]:


#show off the function using a small portion of the data
res2 = appendClosestGeneInfo(x=res[ix.sig,], chr=2, pos=5, genome="hg38") 

#if you get an error, try running on a single thread
#res2 = appendClosestGeneInfo(x=res[ix.sig,], chr=2, pos=5, genome="hg38", numCores=1) 

#if you still get an error, there may be issues with the web host(see below)

head(res2, 50)
write.csv(file="deAnno_padj0.05_CtrlVsDrug.txt", x=res2, row.names=F)

In [ ]:
#################################
#I've run into intermittent problems with the code above.
#the original site that hosts the reference database occasionally goes offline
#If you still want to use the code above, you can hack some of the functions inside
#the ACME package to make it point to a different host


# mygetRefflat <- function (genome = "hg38"){
#     tmpfile <- tempfile()
#     download.file(paste("http://hgdownload.soe.ucsc.edu/goldenPath/", 
#         genome, "/database/refFlat.txt.gz", sep = ""), tmpfile, 
#         mode = "wb")
#     rf <- read.delim(conn <- gzfile(tmpfile, open = "rt"), header = FALSE, 
#         sep = "\t")
#     close(conn)
#     colnames(rf) <- c("geneName", "name", "chrom", "strand", 
#         "txStart", "txEnd", "cdsStart", "cdsEnd", "exonCount", 
#         "exonStarts", "exonEnds")
#     txEndNeg <- rf$txStart
#     txStartNeg <- rf$txEnd
#     cdsStartNeg <- rf$cdsEnd
#     cdsEndNeg <- rf$cdsStart
#     NegStrand <- rf$strand == "-"
#     rf[NegStrand, "cdsEnd"] <- cdsEndNeg[NegStrand]
#     rf[NegStrand, "cdsStart"] <- cdsStartNeg[NegStrand]
#     rf[NegStrand, "txEnd"] <- txEndNeg[NegStrand]
#     rf[NegStrand, "txStart"] <- txStartNeg[NegStrand]
#     return(rf)
# }



In [ ]:
# myfindClosestGene <- function (chrom, pos, genome = "hg38", position = "txStart"){
#     if (!exists("refflat")) {
#         reftmp <- list()
#         reftmp[[genome]] <- mygetRefflat(genome)
#         assign("refflat", reftmp, .GlobalEnv)
#     }
#     else if (!(genome %in% names(refflat))) {
#         refflat[[genome]] <<- mygetRefflat(genome)
#     }
#     rf <- refflat[[genome]]
#     chromsub <- rf$chrom == chrom
#     diffdist <- rf[chromsub, position] - pos
#     sub <- which(abs(diffdist) == min(abs(diffdist)))
#     rf <- rf[chromsub, 1:9][sub, ]
#     return(data.frame(rf, Distance = diffdist[sub]))
# }

# #now you can try again
# appendClosestGeneInfo2 <- function(x, chr=1, pos = 2, genome="hg38", numCores = 4){
# 	res = mclapply(1:nrow(x), function(i){
# 		if (i %% 1000 == 0){
# 			print(i)
# 		}
# 		genes = myfindClosestGene(x[i,chr],x[i, pos],genome)
# 		#if there's more than one, just take the first
# 		data.frame(genes[1,])
# 	}, mc.cores = numCores)

# 	x.res = bind_rows(res)
# 	cbind(x.res, x)
# }


# res2 = appendClosestGeneInfo2(x=res[ix.sig,], chr=2, pos=5, genome="hg38") 
# head(res2, 50)
# write.csv(file="deAnno_padj0.05_CtrlVsDrug.txt", x=res2, row.names=F)


## Visualizing peaks

Once the interesting regions are found with DESeq2, [IGV](https://www.igv.org/) 
can be used to show off the differences visually.
The aligned read files(.bam),can be loaded directly into IGV, or you can
create coverage files(.tdf) with drag/drop tools from within IGV.
The .tdfs are much smaller than .bams but only contain info on the read depths
averaged over a sliding window and are normalized by library size.

If you install IGV on your own computer, you can access its command line
tools which allow you batch process multiple files at once.

IGV comes preloaded with many different reference genomes, so make sure
you select the same one that was used to do your own alignment or your plots
won't make sense.

If you're not well versed in R, you may feel more comfortable working through your
DE results in excel, sorting by the adjusted p-value and filtering by logFC and/or distance
to TSS.  If you're using the `runDE(...)` function defined above, the very last column of the result file 
will have the **chromosomal position** of the peak region in the form *chr:start-stop*.  This specific
format is used in many downstream tools, including IGV, and can be copy/pasted into the location field to
navigate to interesting regions quickly.

![](data/igv_snapshot1.png)